In [1]:
%pip install flask pyngrok transformers scipy
from flask import Flask, request, send_file, make_response
from pyngrok import ngrok
import time
from transformers import pipeline
import scipy.io.wavfile
import os
import getpass

app = Flask(__name__)
authtoken = getpass.getpass("Ingresá tu NGROK_AUTHTOKEN (no se mostrará): ")
ngrok.set_auth_token(authtoken)
public_url = ngrok.connect(5000)
print(f'El servidor está disponible públicamente en: {public_url}')

@app.route("/generate_music", methods=["POST", "OPTIONS"])
def generate_music():
    if request.method == 'OPTIONS':
          response = make_response()
          response.headers.add("Access-Control-Allow-Origin", "*")
          response.headers.add("Access-Control-Allow-Methods", "GET, POST, OPTIONS")
          response.headers.add("Access-Control-Allow-Headers", "Content-Type, Authorization")
          return response
    output_path = "output/musicgen_out.wav"
    prompt = request.json.get("prompt", "Default melody")
    start_time = time.time()

    try:
        print("Cargando modelo...")
        synthesiser = pipeline("text-to-audio", "facebook/musicgen-small")
        print("Modelo cargado. Generando música...")
        music = synthesiser(prompt, forward_params={"do_sample": True})
        scipy.io.wavfile.write(output_path, rate=music["sampling_rate"], data=music["audio"])
        print("Audio generado en %.2f segundos" % (time.time() - start_time))
        print(music, 'Audio generado en: ', output_path)

        #Generar un registro del prompt con los atributos: prompt, fecha, duracion de la pista
        registro_prompt = {
            "prompt": "f{prompt}",
            "fecha_introduccion": "2024-06-11T15:30:00Z",
            "duracion_pista_segundos": 120
        }

        if os.path.exists(output_path):
            return send_file(output_path, as_attachment=True), registro_prompt
        else:
            return "Error: No se pudo generar el archivo de audio.", 500
    except Exception as e:
        print(f"Error: {e}")
        return f"Error al generar la música: {str(e)}", 500

if not os.path.exists("output"):
    os.makedirs("output")

app.run(port=5000)

Ingresá tu NGROK_AUTHTOKEN (no se mostrará): ··········
El servidor está disponible públicamente en: NgrokTunnel: "https://f9d6-34-34-77-83.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Cargando modelo...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/7.87k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0


Modelo cargado. Generando música...


`torch.nn.functional.scaled_dot_product_attention` does not support having an empty attention mask. Falling back to the manual attention implementation. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.Note that this probably happens because `guidance_scale>1` or because you used `get_unconditional_inputs`. See https://github.com/huggingface/transformers/issues/31189 for more information.
INFO:werkzeug:127.0.0.1 - - [07/Jun/2025 22:05:03] "POST /generate_music HTTP/1.1" 200 -


Audio generado en 88.97 segundos
{'audio': array([[[ 0.03051475,  0.0302033 ,  0.0302553 , ..., -0.00219517,
         -0.00157095, -0.0033554 ]]], dtype=float32), 'sampling_rate': 32000} Audio generado en:  output/musicgen_out.wav
